## ElasticSearch

In [ ]:
!pip install elasticsearch
!pip install utm

In [21]:
from elasticsearch import Elasticsearch
import os 
 
url = os.environ['URL']
cloud_id = os.environ['CLOUD_ID'] 

es = Elasticsearch(
    cloud_id = cloud_id,
    basic_auth=('admin', 'admin1234'),
)

In [44]:
# ignore 400 cause by IndexAlreadyExistsException when creating an index
es.indices.create(index='trafico-madrid', ignore=400)

C:\Users\Daniel\AppData\Local\Temp\ipykernel_2264\1222550166.py:2: DeprecationWarning: Passing transport options in the API method is deprecated. Use 'Elasticsearch.options()' instead.
  es.indices.create(index='trafico-madrid', ignore=400)


ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'trafico-madrid'})

In [43]:
es.indices.delete(index='trafico-madrid', ignore=[400, 404])

C:\Users\Daniel\AppData\Local\Temp\ipykernel_2264\3282445592.py:1: DeprecationWarning: Passing transport options in the API method is deprecated. Use 'Elasticsearch.options()' instead.
  es.indices.delete(index='trafico-madrid', ignore=[400, 404])


ObjectApiResponse({'acknowledged': True})

In [45]:
import requests
import json

username = 'admin'
password = 'admin1234'

index = 'trafico-madrid'
headers = {"Content-Type": "application/json"}

## ElasticSearch 1.3 

Utiliza una request de tipo PUT para crear un índice llamado “trafico‐madrid” que esté dividido en 3 shards y en la que cada shard tenga una única copia.  

In [46]:
index = 'trafico-madrid'
data= """{
  "settings": {
    "number_of_shards": 3,
    "number_of_replicas": 1
  }
}"""

response = requests.put(url+index, data=data, headers=headers, auth=(username, password))
res = response.json()
print(res)

{'error': {'root_cause': [{'type': 'resource_already_exists_exception', 'reason': 'index [trafico-madrid/3PyIz7HuRoq3GJDLkUrE2w] already exists', 'index_uuid': '3PyIz7HuRoq3GJDLkUrE2w', 'index': 'trafico-madrid'}], 'type': 'resource_already_exists_exception', 'reason': 'index [trafico-madrid/3PyIz7HuRoq3GJDLkUrE2w] already exists', 'index_uuid': '3PyIz7HuRoq3GJDLkUrE2w', 'index': 'trafico-madrid'}, 'status': 400}


Actualiza  las settings  del  índice  añadiendo  una  nueva setting  que  limite  el  número 
máximo de campos a 40: 

In [47]:
index = 'trafico-madrid/_settings'
data= """{
  "settings": {
    "index.mapping.total_fields.limit": 40
  }
}"""

response = requests.put(url+index, data=data, headers=headers, auth=(username, password))

In [48]:
res = response.json()
print(res)

{'acknowledged': True}


Utiliza  una  request  de  tipo  GET  para  obtener  las settings  del  índice  y  verificar  que 
están correctamente configuradas 

In [49]:
index = 'trafico-madrid/_settings'

response = requests.get(url+index, headers=headers, auth=(username,password))
res = response.json()
print(res)

{'trafico-madrid': {'settings': {'index': {'routing': {'allocation': {'include': {'_tier_preference': 'data_content'}}}, 'mapping': {'total_fields': {'limit': '40'}}, 'number_of_shards': '1', 'provided_name': 'trafico-madrid', 'creation_date': '1712676114644', 'number_of_replicas': '1', 'uuid': '3PyIz7HuRoq3GJDLkUrE2w', 'version': {'created': '8503000'}}}}}


De  acuerdo  con  la  estructura  de  datos  del  fichero  XML  con  el  tráfico  a  tiempo  real, 
crea un mapping que sea capaz de almacenar ese modelo de datos. Recuerda que la 
creación  de  un  mapping  requiere  una  request  de  tipo  PUT.  Ten  presente  que  a  los 
datos  de  cada  elemento  <PM>  del  XML  tendrás  que  añadirle  el  elemento 
<FECHA_HORA> para tener un timestamp de los valores de tráfico. 

In [50]:
index = 'trafico-madrid'


es.indices.delete(index=index)

# Definición del mapping actualizado
mapping = {
    "mappings": {
        "properties": {
            "fecha_hora": {
                "type": "date",
                "format": "yyyy-MM-dd'T'HH:mm:ss'Z'"
            },
            "idelem": {
                "type": "keyword"
            },
            "descripcion": {
                "type": "text"
            },
            "accesoAsociado": {
                "type": "keyword"
            },
            "intensidad": {
                "type": "integer"
            },
            "ocupacion": {
                "type": "integer"
            },
            "carga": {
                "type": "integer"
            },
            "nivelServicio": {
                "type": "keyword"
            },
            "intensidadSat": {
                "type": "integer"
            },
            "error": {
                "type": "keyword"
            },
            "subarea": {
                "type": "keyword"
            },
            "coordenadas": {
                "type": "geo_point"
            }
        }
    }
}

# Indice con el mapping
es.indices.create(index=index, body=mapping)


ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'trafico-madrid'})

In [51]:
#Comprobación

response = requests.get(url+index, headers=headers, auth=(username,password))
res = response.json()
print(res)

{'trafico-madrid': {'aliases': {}, 'mappings': {'properties': {'accesoAsociado': {'type': 'keyword'}, 'carga': {'type': 'integer'}, 'coordenadas': {'type': 'geo_point'}, 'descripcion': {'type': 'text'}, 'error': {'type': 'keyword'}, 'fecha_hora': {'type': 'date', 'format': "yyyy-MM-dd'T'HH:mm:ss'Z'"}, 'idelem': {'type': 'keyword'}, 'intensidad': {'type': 'integer'}, 'intensidadSat': {'type': 'integer'}, 'nivelServicio': {'type': 'keyword'}, 'ocupacion': {'type': 'integer'}, 'subarea': {'type': 'keyword'}}}, 'settings': {'index': {'routing': {'allocation': {'include': {'_tier_preference': 'data_content'}}}, 'number_of_shards': '1', 'provided_name': 'trafico-madrid', 'creation_date': '1712676118645', 'number_of_replicas': '1', 'uuid': 'We1zq7MzSHSeDy6Ggdhc2w', 'version': {'created': '8503000'}}}}}


Realiza una request de tipo POST para añadir un documento de prueba manualmente 
y verificar que se ingesta correctamente y que el mapping no da problemas. Puedes 
usar el siguiente formato:

In [52]:
index = 'trafico-madrid/_doc'
data = """{
    "fecha_hora":"2015-01-01T12:10:30Z",
    "idelem":"11111",
    "descripcion": "Calle Falsa 123. Sentido N-S",
    "accesoAsociado": "222222",
    "intensidad":100,
    "ocupacion":1,
    "carga":5,
    "nivelServicio":0,
    "intensidadSat":3000,
    "error":"S",
    "subarea":"2222",
    "coordenadas":{
        "lat": 41.12,
        "lon": -71.34
    }
}"""

response = requests.post(url+index, data=data, headers=headers, auth=(username, password))
res = response.json()
print(res)

{'_index': 'trafico-madrid', '_id': 'eVNzw44B0_2AQHOVo5nP', '_version': 1, 'result': 'created', '_shards': {'total': 2, 'successful': 2, 'failed': 0}, '_seq_no': 0, '_primary_term': 1}


Haz una búsqueda para recuperar ese documento y ver si se ha almacenado 
correctamente y comprobar el _ID que le ha asignado Elasticsearch.  

In [53]:
index = "trafico-madrid/_search"
data = """
{
  "query": {
    "match": {
      "idelem": "11111"
    }
  }
}
"""

response = requests.get(url+index, headers=headers, data=data, auth=(username,password))
res = response.json()
print(res)

{'took': 2, 'timed_out': False, '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0}, 'hits': {'total': {'value': 1, 'relation': 'eq'}, 'max_score': 0.2876821, 'hits': [{'_index': 'trafico-madrid', '_id': 'eVNzw44B0_2AQHOVo5nP', '_score': 0.2876821, '_source': {'fecha_hora': '2015-01-01T12:10:30Z', 'idelem': '11111', 'descripcion': 'Calle Falsa 123. Sentido N-S', 'accesoAsociado': '222222', 'intensidad': 100, 'ocupacion': 1, 'carga': 5, 'nivelServicio': 0, 'intensidadSat': 3000, 'error': 'S', 'subarea': '2222', 'coordenadas': {'lat': 41.12, 'lon': -71.34}}}]}}


ID del documento: hVOQn44B0_2AQHOVezXJ

Una vez verificado, bórralo utilizando ese _ID

In [54]:
index = "trafico-madrid/_doc/hVOQn44B0_2AQHOVezXJ"

response = requests.delete(url+index, headers=headers, auth=(username,password))
res = response.json()
print(res)

{'_index': 'trafico-madrid', '_id': 'hVOQn44B0_2AQHOVezXJ', '_version': 1, 'result': 'not_found', '_shards': {'total': 2, 'successful': 2, 'failed': 0}, '_seq_no': 1, '_primary_term': 1}


In [55]:
# Comprobación

index = "trafico-madrid/_search"
data = """
{
  "query": {
    "match": {
      "idelem": "11111"
    }
  }
}
"""

response = requests.get(url+index, headers=headers, data=data, auth=(username,password))
res = response.json()
print(res)

{'took': 0, 'timed_out': False, '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0}, 'hits': {'total': {'value': 1, 'relation': 'eq'}, 'max_score': 0.2876821, 'hits': [{'_index': 'trafico-madrid', '_id': 'eVNzw44B0_2AQHOVo5nP', '_score': 0.2876821, '_source': {'fecha_hora': '2015-01-01T12:10:30Z', 'idelem': '11111', 'descripcion': 'Calle Falsa 123. Sentido N-S', 'accesoAsociado': '222222', 'intensidad': 100, 'ocupacion': 1, 'carga': 5, 'nivelServicio': 0, 'intensidadSat': 3000, 'error': 'S', 'subarea': '2222', 'coordenadas': {'lat': 41.12, 'lon': -71.34}}}]}}


## ElasticSearch 1.4

In [56]:
from xml.etree import ElementTree

response = requests.get("https://informo.madrid.es/informo/tmadrid/pm.xml")
root = ElementTree.fromstring(response.content)

fecha = root[0].text
print(fecha)

09/04/2024 17:15:09


In [57]:
out = []
for pm in root[1:-1]:
    d={}
    for i in pm:
        d[i.tag] = i.text
    out.append(d)

In [58]:
print("Hay ", len(out), " medidas en el XML")

Hay  4768  medidas en el XML


In [59]:
out[4767]

{'idelem': '6808',
 'intensidad': '300',
 'ocupacion': '3',
 'carga': '25',
 'nivelServicio': '0',
 'velocidad': '49',
 'error': 'N',
 'st_x': '441891,868185106',
 'st_y': '4481354,05712662'}

In [17]:
import pandas as pd

In [61]:
df=pd.DataFrame(out)
df.head()

,idelem,descripcion,accesoAsociado,intensidad,ocupacion,carga,nivelServicio,intensidadSat,error,subarea,st_x,st_y,velocidad
0,3409,SEPULVEDA Ø118 N-S (CEBREROS-CJAL. FCO. J. JIM...,240102,40,1,2,0,3000,N,1718,"436004,103195086","4472588,47356732",NaN
1,4739,CJAL. FCO. J. JIMENEZ Ø126 E-O (BERLANAS-SEPUL...,240101,280,13,29,1,1500,N,1718,"436039,137988103","4472395,05226989",NaN
2,4740,CJAL. FCO. J. JIMENEZ Ø86 O-E (F. CALVO-ALHAMBRA),240604,315,4,24,0,1500,N,1712,"436730,451619122","4472483,85415038",NaN
3,4741,CJAL. FCO. J. JIMENEZ Ø76 E-O (HURTUMPASCUAL-A...,240602,180,1,13,0,1500,N,1712,"436895,947727199","4472512,33474052",NaN
4,4742,ALHAMBRA Ø23 N-S (CJAL. FCO. J. JIMENEZ -CUART...,241101,450,2,17,0,3000,N,1712,"436809,990969673","4472410,09421372",NaN


In [62]:
df["velocidad"].unique()

array([nan, '67', '70', '59', '72', '75', '78', '91', '84', '58', '81',
       '77', '95', '76', '71', '-1', '39', '26', '42', '47', '73', '65',
       '40', '88', '74', '68', '54', '51', '57', '79', '80', '69', '63',
       '45', '62', '83', '94', '89', '64', '86', '90', '87', '85', '16',
       '56', '92', '43', '82', '48', '19', '17', '66', '46', '22', '60',
       '55', '102', '96', '53', '37', '44', '50', '33', '15', '99', '20',
       '93', '49', '12', '18', '7', '28', '61', '52', '13'], dtype=object)

In [63]:
df.columns

Index(['idelem', 'descripcion', 'accesoAsociado', 'intensidad', 'ocupacion',
       'carga', 'nivelServicio', 'intensidadSat', 'error', 'subarea', 'st_x',
       'st_y', 'velocidad'],
      dtype='object')

In [60]:
import datetime
import utm
from unidecode import unidecode
import time

index = "trafico-madrid"

def convert_to_utf8(s):
    return unidecode(s.replace("®", "n.").replace("*", "").replace("'", "'"))

def procesar_data(out):
    
    data_procesada_list = []
    
    start=time.time()
    
    for i in out:

        try:
            i["idelem"] = convert_to_utf8(i["idelem"])
        except KeyError:
            pass
        except:
            del i["idelem"]
        try:
            i["descripcion"] = convert_to_utf8(i["descripcion"])
        except KeyError:
            pass
        except:
            del i["descripcion"]
        try:
            i["accesoAsociado"] = convert_to_utf8(i["accesoAsociado"])
        except KeyError:
            pass
        except:
            del i["accesoAsociado"]
        try:
            intensidad = float(i["intensidad"])
            i["intensidad"]=intensidad
        except KeyError:
            pass
        except:
            del i["intensidad"]
        try:
            i["ocupacion"] = float(i["ocupacion"])
        except KeyError:
            pass
        except:
            del i["ocupacion"]
        try:
            i["carga"] = float(i["carga"])
        except KeyError:
            pass
        except:
            del i["carga"]
        try:
            i["intensidadSat"] = float(i["intensidadSat"])
        except KeyError:
            pass
        except:
            del i["intensidadSat"]
        try:
            i["nivelServicio"] = float(i["nivelServicio"])
        except KeyError:
            pass
        except:
            del i["nivelServicio"]
        try:
            i["error"] = convert_to_utf8(i["error"])
        except KeyError:
            pass
        except:
            del i["error"]
        try:
            i["subarea"] = convert_to_utf8(i["subarea"])
        except KeyError:
            pass
        except:
            del i["subarea"]

        # Coordenadas
        latitud, longitud = utm.to_latlon(float(i["st_x"].replace(',', '.')), float(i["st_y"].replace(',', '.')), 30, 'T')
        i["coordenadas"] = {"lat": latitud, "lon": longitud}

        data_procesada_list.append(i)
        data = str(i).replace("'", "\"")
        
        try:
            response = es.index(index=index, body=data)
        except Exception as e:
            print(f"Error al ingestar la medida: {e}")
        
    end = time.time()
    totaltime = end - start

    print("tiempo total:", totaltime)
    return data_procesada_list

In [61]:
data_procesada = procesar_data(out) # Esto tardará un rato

Error al ingestar la medida: BadRequestError(400, 'document_parsing_exception', '[1:35] failed to parse: [1:67] Unexpected character (\'D\' (code 68)): was expecting comma to separate Object entries\n at [Source: (byte[])"{"idelem": "6453", "descripcion": "Arroyo Media Legua - Acceso O"Donnell", "accesoAsociado": "5233002", "intensidad": 420.0, "ocupacion": 7.0, "carga": 36.0, "nivelServicio": 1.0, "intensidadSat": 1200.0, "error": "N", "subarea": "3501", "st_x": "444667,573510715", "st_y": "4473918,86976192", "coordenadas": {"lat": 40.41405302543771, "lon": -3.6521805245344026}}"; line: 1, column: 67]')
Error al ingestar la medida: BadRequestError(400, 'document_parsing_exception', '[1:35] failed to parse: [1:46] Unexpected character (\'D\' (code 68)): was expecting comma to separate Object entries\n at [Source: (byte[])"{"idelem": "6457", "descripcion": "Acceso O"Donnell - Acceso Arroyo Media Legua desde O"Donnell", "accesoAsociado": "5234004", "intensidad": 380.0, "ocupacion": 5.0, 

Error al ingestar la medida: BadRequestError(400, 'document_parsing_exception', '[1:35] failed to parse: [1:68] Unexpected character (\'D\' (code 68)): was expecting comma to separate Object entries\n at [Source: (byte[])"{"idelem": "4067", "descripcion": "Narvaez S-N - Doctor Castelo-O"Donnell", "accesoAsociado": "1005001", "intensidad": 180.0, "ocupacion": 2.0, "carga": 26.0, "nivelServicio": 1.0, "intensidadSat": 800.0, "error": "N", "subarea": "0101", "st_x": "442647,643706406", "st_y": "4474655,63772956", "coordenadas": {"lat": 40.42055347811375, "lon": -3.6760534934167413}}"; line: 1, column: 68]')
Error al ingestar la medida: BadRequestError(400, 'document_parsing_exception', '[1:35] failed to parse: [1:74] Unexpected character (\'D\' (code 68)): was expecting comma to separate Object entries\n at [Source: (byte[])"{"idelem": "4068", "descripcion": "(AFOROS) Av. Menendez Pelayo N-S - O"Donnell-Doctor Castelo", "accesoAsociado": "1011002", "intensidad": 760.0, "ocupacion": 4.0, "

In [62]:
# Comprobación de que se ha guardado correctamente (Mas información en el informe)

index = "trafico-madrid/_search"
data = """
{
  "query": {
    "match": {
      "idelem": "11260"
    }
  }
}
"""

response = requests.get(url+index, headers=headers, data=data, auth=(username,password))
res = response.json()
print(res)

{'took': 0, 'timed_out': False, '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0}, 'hits': {'total': {'value': 1, 'relation': 'eq'}, 'max_score': 8.058748, 'hits': [{'_index': 'trafico-madrid', '_id': 'lVN4w44B0_2AQHOVNKKI', '_score': 8.058748, '_source': {'idelem': '11260', 'descripcion': 'Suecia, 42 - Bucarest-Budapest', 'accesoAsociado': '6607001', 'intensidad': 400.0, 'ocupacion': 10.0, 'carga': 22.0, 'nivelServicio': 0.0, 'intensidadSat': 1800.0, 'error': 'N', 'subarea': '3225', 'st_x': '448735,587714078', 'st_y': '4475242,54177595', 'coordenadas': {'lat': 40.42623802369264, 'lon': -3.604341844158264}}}]}}


## ElasticSearch 1.5

In [63]:
index = 'trafico-madrid/_mappings'

response = requests.get(url+index, headers=headers, auth=(username,password))
res = response.json()
print(res)

{'trafico-madrid': {'mappings': {'properties': {'accesoAsociado': {'type': 'keyword'}, 'carga': {'type': 'integer'}, 'coordenadas': {'type': 'geo_point'}, 'descripcion': {'type': 'text'}, 'error': {'type': 'keyword'}, 'fecha_hora': {'type': 'date', 'format': "yyyy-MM-dd'T'HH:mm:ss'Z'"}, 'idelem': {'type': 'keyword'}, 'intensidad': {'type': 'integer'}, 'intensidadSat': {'type': 'integer'}, 'nivelServicio': {'type': 'keyword'}, 'ocupacion': {'type': 'integer'}, 'st_x': {'type': 'text', 'fields': {'keyword': {'type': 'keyword', 'ignore_above': 256}}}, 'st_y': {'type': 'text', 'fields': {'keyword': {'type': 'keyword', 'ignore_above': 256}}}, 'subarea': {'type': 'keyword'}, 'velocidad': {'type': 'text', 'fields': {'keyword': {'type': 'keyword', 'ignore_above': 256}}}}}}}


In [90]:
df=pd.DataFrame(data_procesada)
df.head()

,idelem,descripcion,accesoAsociado,intensidad,ocupacion,carga,nivelServicio,intensidadSat,error,subarea,st_x,st_y,coordenadas
0,3409,SEPULVEDA n.118 N-S (CEBREROS-CJAL. FCO. J. JI...,240102,127.0,1.0,5.0,0,3000.0,N,1718,"436004,103195086","4472588,47356732","{'lat': 40.401447264855946, 'lon': -3.75415167..."
1,4739,CJAL. FCO. J. JIMENEZ n.126 E-O (BERLANAS-SEPU...,240101,339.0,6.0,27.0,1,1500.0,N,1718,"436039,137988103","4472395,05226989","{'lat': 40.399707541055946, 'lon': -3.75371940..."
2,4740,CJAL. FCO. J. JIMENEZ n.86 O-E (F. CALVO-ALHAM...,240604,90.0,1.0,7.0,0,1500.0,N,1712,"436730,451619122","4472483,85415038","{'lat': 40.40056031741004, 'lon': -3.745582402..."
3,4741,CJAL. FCO. J. JIMENEZ n.76 E-O (HURTUMPASCUAL-...,240602,90.0,0.0,6.0,0,1500.0,N,1712,"436895,947727199","4472512,33474052","{'lat': 40.40082944059434, 'lon': -3.743635138..."
4,4742,ALHAMBRA n.23 N-S (CJAL. FCO. J. JIMENEZ -CUAR...,241101,270.0,1.0,9.0,0,3000.0,N,1712,"436809,990969673","4472410,09421372","{'lat': 40.39990189695159, 'lon': -3.744637846..."


## ElasticSearch 1.6

In [64]:
def ndjsonify(dp):
    ndjson = ""
    for doc in dp:
        
        ndjson += '{ "index": {"_index" : "trafico-madrid"}}\n'
        ndjson += json.dumps(doc) + '\n'
    
    return ndjson

In [65]:
index = 'trafico-madrid'
api = """/_bulk""" # Hay que quitar el campo "_doc", sino el request dará error (https://stackoverflow.com/questions/74597731/a-mapper-parsing-exception-occurred-when-using-the-bulk-api-of-elasticsearch)

ndjson = ndjsonify(data_procesada)

print(ndjson)

{ "index": {"_index" : "trafico-madrid"}}
{"idelem": "3409", "descripcion": "SEPULVEDA O118 N-S (CEBREROS-CJAL. FCO. J. JIMENEZ)", "accesoAsociado": "240102", "intensidad": 254.0, "ocupacion": 1.0, "carga": 9.0, "nivelServicio": 0.0, "intensidadSat": 3000.0, "error": "N", "subarea": "1718", "st_x": "436004,103195086", "st_y": "4472588,47356732", "coordenadas": {"lat": 40.401447264855946, "lon": -3.7541516712567753}}
{ "index": {"_index" : "trafico-madrid"}}
{"idelem": "4739", "descripcion": "CJAL. FCO. J. JIMENEZ O126 E-O (BERLANAS-SEPULVEDA)", "accesoAsociado": "240101", "intensidad": 254.0, "ocupacion": 2.0, "carga": 19.0, "nivelServicio": 0.0, "intensidadSat": 1500.0, "error": "N", "subarea": "1718", "st_x": "436039,137988103", "st_y": "4472395,05226989", "coordenadas": {"lat": 40.399707541055946, "lon": -3.7537194086223162}}
{ "index": {"_index" : "trafico-madrid"}}
{"idelem": "4740", "descripcion": "CJAL. FCO. J. JIMENEZ O86 O-E (F. CALVO-ALHAMBRA)", "accesoAsociado": "240604", "i

In [67]:
start=time.time()

response = requests.post(url+index+api, data=ndjson, headers=headers, auth=(username,password))
res = response.json()
print(res)

end = time.time()
totaltime = end - start

print(totaltime)

{'errors': False, 'took': 309, 'items': [{'index': {'_index': 'trafico-madrid', '_id': 'v1OKw44B0_2AQHOVFb4P', '_version': 1, 'result': 'created', '_shards': {'total': 2, 'successful': 2, 'failed': 0}, '_seq_no': 9510, '_primary_term': 1, 'status': 201}}, {'index': {'_index': 'trafico-madrid', '_id': 'wFOKw44B0_2AQHOVFb4P', '_version': 1, 'result': 'created', '_shards': {'total': 2, 'successful': 2, 'failed': 0}, '_seq_no': 9511, '_primary_term': 1, 'status': 201}}, {'index': {'_index': 'trafico-madrid', '_id': 'wVOKw44B0_2AQHOVFb4P', '_version': 1, 'result': 'created', '_shards': {'total': 2, 'successful': 2, 'failed': 0}, '_seq_no': 9512, '_primary_term': 1, 'status': 201}}, {'index': {'_index': 'trafico-madrid', '_id': 'wlOKw44B0_2AQHOVFb4P', '_version': 1, 'result': 'created', '_shards': {'total': 2, 'successful': 2, 'failed': 0}, '_seq_no': 9513, '_primary_term': 1, 'status': 201}}, {'index': {'_index': 'trafico-madrid', '_id': 'w1OKw44B0_2AQHOVFb4P', '_version': 1, 'result': 'cre